In [1]:
import pandas as pd
import numpy as np
import os


In [15]:

def preprocess_snow_data(folder_path):
    all_data = []

    # Get all CSV files in the folder
    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

    for file_path in file_paths:
        # Read the first few rows to extract the year
        with open(file_path, 'r') as file:
            first_row = file.readlines()[0]  # Get the first row
            year_month = first_row.split(',')[1].strip()  # Extract the month-year string
            year = year_month.split(' ')[1]  # Split and take the year part

        # Load the actual data, skipping the metadata rows
        df = pd.read_csv(file_path, skiprows=4)

        # Identify date columns by removing metadata-related columns
        meta_cols = ["GHCN ID", "Station Name", "County", "State", "Elevation", "Latitude", "Longitude"]
        date_cols = [col for col in df.columns if col not in meta_cols]

        # Melt the dataframe to long format, keeping only date columns
        df_long = df.melt(id_vars=["County"], value_vars=date_cols, var_name="date", value_name="snow_depth")

        # Add the year to the 'date' column
        df_long['date'] = df_long['date'] + ' ' + year  # Append the extracted year to the date

        # Replace 'M' with NaN and 'T' (trace amounts) with 0.01
        df_long["snow_depth"] = df_long["snow_depth"].replace({"M": None, "T": 0.01}).astype(float)

        # Append to the list
        all_data.append(df_long)

    # Concatenate all files
    clean_data = pd.concat(all_data, ignore_index=True)

    # Convert 'date' to datetime
    clean_data['date'] = pd.to_datetime(clean_data['date'], format='%b %d %Y')

    return clean_data

# Example: preprocess data from a folder
folder_path = '.\data'
cleaned_snow_data = preprocess_snow_data(folder_path)
cleaned_snow_data.head()


,County,date,snow_depth
0,STILLWATER,2015-05-01,0.0
1,MINERAL,2015-05-01,0.0
2,CARTER,2015-05-01,0.0
3,MADISON,2015-05-01,0.0
4,DEERLODGE,2015-05-01,0.0


In [16]:
def aggregate_snow_data(snow_data, county_areas):
    
    # Step 1: Average sensor readings per county per day
    county_avg_snow = snow_data.groupby(["date", "County"])["snow_depth"].mean().reset_index()

    # Step 2: Merge with county area data
    county_avg_snow = county_avg_snow.merge(county_areas, on="County", how="left")

    # Step 3: Compute the weighted average snow depth for the state
    county_avg_snow["weighted_snow"] = county_avg_snow["snow_depth"] * county_avg_snow["AREA"]
    
    # Compute the state-wide weighted average per day
    state_snow = county_avg_snow.groupby("date").apply(
        lambda x: x["weighted_snow"].sum() / x["AREA"].sum()
    ).reset_index(name="state_avg_snow")
    
    # Ensure the result is in DataFrame format
    return state_snow

# Load county area data
county_areas_df = pd.read_csv("MTcounties.csv")

# Example usage (assuming cleaned data is ready)
aggregated_snow_data = aggregate_snow_data(cleaned_snow_data, county_areas_df)
aggregated_snow_data = aggregated_snow_data.sort_values(by="date")

# Print or work with the DataFrame
print(aggregated_snow_data)
aggregated_snow_data.to_csv("cleaned.csv")


           date  state_avg_snow
0    2015-01-01        0.000386
1    2015-01-02        0.175324
2    2015-01-03        1.424501
3    2015-01-04        1.132926
4    2015-01-05        2.866150
...         ...             ...
3679 2025-01-27        0.000000
3680 2025-01-28        0.000000
3681 2025-01-29        0.000042
3682 2025-01-30        0.018836
3683 2025-01-31        0.036596

[3684 rows x 2 columns]


In [13]:
# Define the folder containing the CSV files
folder_path = ".\montana_data_14_743_GoodStations"

# Define the required columns
required_columns = ["STATION", "DATE", "NAME", "TMAX", "TMIN", "RHMN", "RHMX"]

# Define the date range
start_date = "2015-01-01"
end_date = "2024-12-31"

# List to store filtered dataframes
filtered_dfs = []

# Loop through each file in the folder
for file in os.listdir(folder_path):
    if file.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(folder_path, file)

        # Read CSV while ensuring the DATE column is parsed as a datetime object
        df = pd.read_csv(file_path, parse_dates=["DATE"], dtype=str)

        # Keep only the required columns (if they exist)
        df = df[required_columns].copy()

        # Filter rows within the date range
        df = df[(df["DATE"] >= start_date) & (df["DATE"] <= end_date)]

        # Append filtered dataframe to the list
        filtered_dfs.append(df)

# Concatenate all filtered dataframes into one (optional)
final_df = pd.concat(filtered_dfs, ignore_index=True)
final_df['DATE'] = pd.to_datetime(final_df['DATE'], format = '%b %d %Y') # Make sure dates are datetime

# Save to a new CSV file (optional)
#final_df.to_csv("filtered_data.csv", index=False)


In [16]:
# Dictionary mapping station names to county names
station_to_county = {
    'BOZEMAN GALLATIN FIELD AIRPORT, MT US': 'GALLATIN',
    'GLASGOW INTERNATIONAL AIRPORT, MT US': 'VALLEY',
    'BUTTE BERT MOONEY AIRPORT, MT US': 'SILVER BOW',
    'CUT BANK AIRPORT, MT US': 'GLACIER',
    'LIVINGSTON AIRPORT, MT US': 'PARK',
    'HELENA AIRPORT ASOS, MT US': 'LEWISANDCLARK',
    'BILLINGS INTERNATIONAL AIRPORT, MT US': 'YELLOWSTONE',
    'MISSOULA INTERNATIONAL AIRPORT, MT US': 'MISSOULA',
    'LEWISTOWN AIRPORT, MT US': 'FERGUS',
    'GREAT FALLS AIRPORT, MT US': 'CASCADE',
    'MILES CITY AIRPORT, MT US': 'CUSTER',
    'KALISPELL GLACIER AIRPORT, MT US': 'FLATHEAD',
    'DILLON AIRPORT, MT US': 'BEAVERHEAD',
    'HAVRE AIRPORT ASOS, MT US': 'HILL'
}

In [17]:
final_df["COUNTY"] = final_df["NAME"].map(station_to_county)
final_df.to_csv("filtered_data.csv", index=False)

In [22]:
def aggregate_weather_data(weather_data, county_areas):
    # Step 1: Average weather readings per county per day
    county_avg_weather = weather_data.groupby(["DATE", "COUNTY"])[["TMAX", "TMIN", "RHMN", "RHMX"]].mean().reset_index()

    # Step 2: Merge with county area data
    county_avg_weather = county_avg_weather.merge(county_areas, on="COUNTY", how="left")

    # Step 3: Compute weighted values
    for col in ["TMAX", "TMIN", "RHMN", "RHMX"]:
        county_avg_weather[f"weighted_{col}"] = county_avg_weather[col] * county_avg_weather["AREA"]

    # Step 4: Compute state-wide weighted averages per day
    state_weather = county_avg_weather.groupby("DATE").apply(
        lambda x: pd.Series({col: x[f"weighted_{col}"].sum() / x["AREA"].sum() for col in ["TMAX", "TMIN", "RHMN", "RHMX"]})
    ).reset_index()

    # Convert DATE to datetime format
    state_weather["DATE"] = pd.to_datetime(state_weather["DATE"])

    # Step 5: Create separate time series DataFrames for each weather variable
    tmax_df = state_weather[["DATE", "TMAX"]].rename(columns={"TMAX": "State_Avg_TMAX"})
    tmin_df = state_weather[["DATE", "TMIN"]].rename(columns={"TMIN": "State_Avg_TMIN"})
    rhmn_df = state_weather[["DATE", "RHMN"]].rename(columns={"RHMN": "State_Avg_RHMN"})
    rhmx_df = state_weather[["DATE", "RHMX"]].rename(columns={"RHMX": "State_Avg_RHMX"})

    # Ensure full date range from 2015-01-01 to 2024-12-31
    full_date_range = pd.date_range(start="2015-01-01", end="2024-12-31", freq="D")
    
    def complete_timeseries(df, column_name):
        df = df.set_index("DATE").reindex(full_date_range).rename_axis("Date").reset_index()
        df = df.rename(columns={"index": "Date", column_name: column_name})
        return df

    tmax_df = complete_timeseries(tmax_df, "State_Avg_TMAX")
    tmin_df = complete_timeseries(tmin_df, "State_Avg_TMIN")
    rhmn_df = complete_timeseries(rhmn_df, "State_Avg_RHMN")
    rhmx_df = complete_timeseries(rhmx_df, "State_Avg_RHMX")

    return tmax_df, tmin_df, rhmn_df, rhmx_df

# Load data
weather_data = pd.read_csv("filtered_data.csv")  # Your cleaned weather dataset with COUNTY column
county_areas = pd.read_csv("MTcounties.csv")  # County area dataset

# Aggregate the data
tmax_series, tmin_series, rhmn_series, rhmx_series = aggregate_weather_data(weather_data, county_areas)

# Save to CSV
tmax_series.to_csv("timeseries\montana_tmax_timeseries.csv", index=False)
tmin_series.to_csv("timeseries\montana_tmin_timeseries.csv", index=False)
rhmn_series.to_csv("timeseries\montana_rhmn_timeseries.csv", index=False)
rhmx_series.to_csv("timeseries\montana_rhmx_timeseries.csv", index=False)

# Print confirmation
print("Aggregated weather data saved as time series for each variable.")


Aggregated weather data saved as time series for each variable.
